In [1]:
%cd G:\내 드라이브\data\health

G:\내 드라이브\data\health


In [165]:
import pandas as pd
import pickle
import numpy as np
import os
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

In [129]:
health = pd.read_csv('health.txt', index_col=0)

C:\Users\Sungmin\AppData\Local\Programs\Python\Python39\lib\site-packages\IPython\core\interactiveshell.py:3457: DtypeWarning: Columns (0) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [46]:
data = health.drop(columns=['YEAR_t', 'trainset'])

In [59]:
data.index = np.arange(0,147472)

In [60]:
data.to_pickle('health.pkl')

In [69]:
with open('health.pkl', 'rb') as f:
    df = pickle.load(f)

In [80]:
df.ClaimsTruncated = df.ClaimsTruncated.astype('int').astype('category')

In [96]:
age = df.apply(lambda x: 0 if x.loc['age_05':'age_65'].sum()==1 else 1, axis=1)

In [97]:
df['age'] = age

In [101]:
df = df.drop(columns=['age_05', 'age_15', 'age_25', 'age_35'
                 , 'age_45', 'age_55', 'age_65', 'age_75'
                 , 'age_85'])

In [109]:
df.DaysInHospital = df.DaysInHospital.apply(lambda x: 0 if x == 0 else 1)

In [134]:
categorical = ['ClaimsTruncated', 'DaysInHospital', 'age_MISS', 'sexMALE', 'sexFEMALE',
               'sexMISS', 'labNull', 'drugNull']

In [141]:
df_categorical = df[categorical]
df_numerical = df.drop(columns = categorical)

In [157]:
hot_categorical = pd.concat([df_categorical[x].astype('int').astype('category') for x in df_categorical], axis=1)
hot_categorical.columns = df_categorical.columns

In [178]:
one_hot_numeric = pd.concat([pd.cut(df_numerical[name], 2, duplicates='drop').astype('category').cat.codes for name in df_numerical], axis=1)
one_hot_numeric.columns = df_numerical.columns

In [179]:
one_hot = pd.concat([hot_categorical.iloc[:, 2:], one_hot_numeric, hot_categorical.iloc[:, 0:2]], axis=1)

In [180]:
%cd C:\Users\Sungmin\PycharmProjects\get_dataset

C:\Users\Sungmin\PycharmProjects\get_dataset


In [181]:
one_hot.rename(columns={'DaysInHospital': 'label'}, inplace=True)

In [183]:
train, test = train_test_split(one_hot, test_size=0.2)
train.index = range(len(train))
test.index = range(len(test))
train.to_pickle('./health/health_train.pkl')
test.to_pickle('./health/health_test.pkl')